In [5]:
import os
import glob
import pandas as pd
import shutil

def process_excel_and_run_foldx(excel_path, foldx_path):
    df = pd.read_excel(excel_path)
    
    for index, row in df.iterrows():
        protein_id = row['PDB ID']
        chain = row['Mutated Chain']
        mutation = row['Mutation_PDB']
        ddg_results = []

        print(f"\nProcessing protein {protein_id} with mutation: {mutation}")
        
        file_name = f"individual_list_{protein_id}_{mutation}.txt"
        file_path = os.path.join(foldx_path, file_name)
        
        with open(file_path, 'w', encoding='utf-8') as file:
            formatted_mutation = f"{mutation[0]}{chain}{mutation[1:]}"
            file.write(f"{formatted_mutation};\n")

        print(f"{file_name} oluşturuldu ve {foldx_path} dizinine kaydedildi.")

        os.chdir(foldx_path)

        structure_dir = os.path.join(foldx_path, "S4038_structures")
        pdb_filename = f"{protein_id}_{chain}.pdb"
        available_pdbs = os.listdir(structure_dir)
        matching_files = [f for f in available_pdbs if f.lower() == pdb_filename.lower()]

        if matching_files:
            pdb_source_path = os.path.join(structure_dir, matching_files[0])
            pdb_dest_path = os.path.join(foldx_path, f"{protein_id}.pdb")  
            if not os.path.exists(pdb_dest_path):
                shutil.copy(pdb_source_path, pdb_dest_path)
                print(f"{pdb_filename} → {protein_id}.pdb olarak kopyalandı.")
        else:
            print(f"HATA: {pdb_filename} S4038_structures içinde bulunamadı!")
            continue  


        repair_pdb = f"{protein_id}_Repair.pdb"
        if not os.path.exists(repair_pdb):
            repair_command = f"foldx_1_20251231.exe --command=RepairPDB --pdb={protein_id}.pdb"
            print("Onarım komutu:", repair_command)
            os.system(repair_command)
        else:
            print(f"{repair_pdb} zaten var, onarım atlanıyor.")

        
        individual_list = f"individual_list_{protein_id}_{mutation}.txt"
        output_prefix = f"{protein_id}_repair_{mutation}"
        build_command = f"foldx_1_20251231.exe --command=BuildModel --pdb={protein_id}_Repair.pdb --mutant-file={individual_list} --output-file={output_prefix}"
        print("BuildModel komutu:", build_command)
        os.system(build_command)


        dif_pattern = os.path.join(foldx_path, f"Dif_*{mutation}*.fxout")
        dif_files = glob.glob(dif_pattern)
        if dif_files:
            dif_file = dif_files[0]
            with open(dif_file, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                for line in lines:
                    line = line.strip()
                    if line and not line.startswith("Pdb") and not line.startswith("FoldX"):
                        try:
                            ddg_value = float(line.split()[1])
                            ddg_results.append((mutation, ddg_value))
                            print(f"ΔΔG of protein {protein_id}'s mutation {mutation}: {ddg_value}")
                            break
                        except (IndexError, ValueError):
                            continue
                else:
                    print(f"{mutation} için ΔΔG değeri okunamadı.")
        else:
            print(f"{mutation} için Dif dosyası bulunamadı.")

        
        results_file = f"{protein_id}_{mutation}_results.txt"
        with open(results_file, 'w', encoding='utf-8') as f:
            f.write(f"Protein: {protein_id}\n")
            f.write(f"Chain: {chain}\n")
            f.write(f"Mutation: {mutation}\n")
            f.write("ΔΔG Value\n")
            for mutation, ddg_value in ddg_results:
                f.write(f"{ddg_value}\n")
        
        print(f"Sonuçlar {results_file} dosyasına kaydedildi.")


if __name__ == "__main__":
    excel_path = r"C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1)\Data 3.xlsx"
    foldx_path = r"C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1)"
    
    process_excel_and_run_foldx(excel_path, foldx_path)



Processing protein 1A7V with mutation: E73H
individual_list_1A7V_E73H.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1A7V_A.pdb → 1A7V.pdb olarak kopyalandı.
Onarım komutu: foldx_1_20251231.exe --command=RepairPDB --pdb=1A7V.pdb
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1A7V_Repair.pdb --mutant-file=individual_list_1A7V_E73H.txt --output-file=1A7V_repair_E73H
ΔΔG of protein 1A7V's mutation E73H: -0.291432
Sonuçlar 1A7V_E73H_results.txt dosyasına kaydedildi.

Processing protein 1A7V with mutation: K97H
individual_list_1A7V_K97H.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1A7V_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1A7V_Repair.pdb --mutant-file=individual_list_1A7V_K97H.txt --output-file=1A7V_repair_K97H
ΔΔG of protein 1A7V's mutation K97H: 0.752104
Sonuçlar 1A7V_K97H_results.txt dosyasına kayded

BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1APS_Repair.pdb --mutant-file=individual_list_1APS_L65V.txt --output-file=1APS_repair_L65V
ΔΔG of protein 1APS's mutation L65V: 4.09875
Sonuçlar 1APS_L65V_results.txt dosyasına kaydedildi.

Processing protein 1APS with mutation: P71A
individual_list_1APS_P71A.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1APS_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1APS_Repair.pdb --mutant-file=individual_list_1APS_P71A.txt --output-file=1APS_repair_P71A
ΔΔG of protein 1APS's mutation P71A: 1.07938
Sonuçlar 1APS_P71A_results.txt dosyasına kaydedildi.

Processing protein 1APS with mutation: P54A
individual_list_1APS_P54A.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1APS_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --p

N60AA için Dif dosyası bulunamadı.
Sonuçlar 1AYE_N60AA_results.txt dosyasına kaydedildi.

Processing protein 1AYE with mutation: Q62AG
individual_list_1AYE_Q62AG.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1AYE_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1AYE_Repair.pdb --mutant-file=individual_list_1AYE_Q62AG.txt --output-file=1AYE_repair_Q62AG
Q62AG için Dif dosyası bulunamadı.
Sonuçlar 1AYE_Q62AG_results.txt dosyasına kaydedildi.

Processing protein 1AYE with mutation: V66AG
individual_list_1AYE_V66AG.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1AYE_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1AYE_Repair.pdb --mutant-file=individual_list_1AYE_V66AG.txt --output-file=1AYE_repair_V66AG
V66AG için Dif dosyası bulunamadı.
Sonuçlar 1AYE_V66AG_results.txt dosyası

ΔΔG of protein 1AZP's mutation V30A: 3.54543
Sonuçlar 1AZP_V30A_results.txt dosyasına kaydedildi.

Processing protein 1AZP with mutation: R25A
individual_list_1AZP_R25A.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1AZP_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1AZP_Repair.pdb --mutant-file=individual_list_1AZP_R25A.txt --output-file=1AZP_repair_R25A
ΔΔG of protein 1AZP's mutation R25A: 2.7039
Sonuçlar 1AZP_R25A_results.txt dosyasına kaydedildi.

Processing protein 1AZP with mutation: W24A
individual_list_1AZP_W24A.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1AZP_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1AZP_Repair.pdb --mutant-file=individual_list_1AZP_W24A.txt --output-file=1AZP_repair_W24A
ΔΔG of protein 1AZP's mutation W24A: 3.01846
Sonuçlar 1AZP_W24A_r

ΔΔG of protein 1B2M's mutation T93A: -0.0290962
Sonuçlar 1B2M_T93A_results.txt dosyasına kaydedildi.

Processing protein 1B2M with mutation: A21M
individual_list_1B2M_A21M.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1B2M_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1B2M_Repair.pdb --mutant-file=individual_list_1B2M_A21M.txt --output-file=1B2M_repair_A21M
ΔΔG of protein 1B2M's mutation A21M: 0.184736
Sonuçlar 1B2M_A21M_results.txt dosyasına kaydedildi.

Processing protein 1B2M with mutation: A21I
individual_list_1B2M_A21I.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
1B2M_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=1B2M_Repair.pdb --mutant-file=individual_list_1B2M_A21I.txt --output-file=1B2M_repair_A21I
ΔΔG of protein 1B2M's mutation A21I: 0.411054
Sonuçlar 1B2M_

...

ΔΔG of protein 6TG7's mutation A97G: 1.11839
Sonuçlar 6TG7_A97G_results.txt dosyasına kaydedildi.

Processing protein 6TG7 with mutation: A98G
individual_list_6TG7_A98G.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6TG7_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6TG7_Repair.pdb --mutant-file=individual_list_6TG7_A98G.txt --output-file=6TG7_repair_A98G
ΔΔG of protein 6TG7's mutation A98G: -0.0588993
Sonuçlar 6TG7_A98G_results.txt dosyasına kaydedildi.

Processing protein 6TG7 with mutation: T112A
individual_list_6TG7_T112A.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6TG7_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6TG7_Repair.pdb --mutant-file=individual_list_6TG7_T112A.txt --output-file=6TG7_repair_T112A
ΔΔG of protein 6TG7's mutation T112A: 3.52769
Sonuçlar 6T

ΔΔG of protein 6UJH's mutation S62A: 1.58813
Sonuçlar 6UJH_S62A_results.txt dosyasına kaydedildi.

Processing protein 6UJH with mutation: D324E
individual_list_6UJH_D324E.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6UJH_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6UJH_Repair.pdb --mutant-file=individual_list_6UJH_D324E.txt --output-file=6UJH_repair_D324E
ΔΔG of protein 6UJH's mutation D324E: 0.65134
Sonuçlar 6UJH_D324E_results.txt dosyasına kaydedildi.

Processing protein 6UJH with mutation: W330F
individual_list_6UJH_W330F.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6UJH_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6UJH_Repair.pdb --mutant-file=individual_list_6UJH_W330F.txt --output-file=6UJH_repair_W330F
ΔΔG of protein 6UJH's mutation W330F: 0.897562
Sonuçla

ΔΔG of protein 6VCU's mutation R57K: 0.723508
Sonuçlar 6VCU_R57K_results.txt dosyasına kaydedildi.

Processing protein 6VCU with mutation: S67E
individual_list_6VCU_S67E.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6VCU_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6VCU_Repair.pdb --mutant-file=individual_list_6VCU_S67E.txt --output-file=6VCU_repair_S67E
ΔΔG of protein 6VCU's mutation S67E: 3.56325
Sonuçlar 6VCU_S67E_results.txt dosyasına kaydedildi.

Processing protein 6VCU with mutation: Q70D
individual_list_6VCU_Q70D.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6VCU_Repair.pdb zaten var, onarım atlanıyor.
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6VCU_Repair.pdb --mutant-file=individual_list_6VCU_Q70D.txt --output-file=6VCU_repair_Q70D
ΔΔG of protein 6VCU's mutation Q70D: 2.20399
Sonuçlar 6VCU_Q70D

ΔΔG of protein 6W4E's mutation L298R: 0.43739
Sonuçlar 6W4E_L298R_results.txt dosyasına kaydedildi.

Processing protein 6W79 with mutation: W31F
individual_list_6W79_W31F.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6W79_A.pdb → 6W79.pdb olarak kopyalandı.
Onarım komutu: foldx_1_20251231.exe --command=RepairPDB --pdb=6W79.pdb
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6W79_Repair.pdb --mutant-file=individual_list_6W79_W31F.txt --output-file=6W79_repair_W31F
ΔΔG of protein 6W79's mutation W31F: 4.62992
Sonuçlar 6W79_W31F_results.txt dosyasına kaydedildi.

Processing protein 6XPP with mutation: F345A
individual_list_6XPP_F345A.txt oluşturuldu ve C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1) dizinine kaydedildi.
6XPP_D.pdb → 6XPP.pdb olarak kopyalandı.
Onarım komutu: foldx_1_20251231.exe --command=RepairPDB --pdb=6XPP.pdb
BuildModel komutu: foldx_1_20251231.exe --command=BuildModel --pdb=6XPP_Repair.pdb -

TypeError: 'float' object is not subscriptable

In [2]:
import os
import pandas as pd

foldx_path = r"C:\Users\asus\OneDrive\Masaüstü\foldx5_1Windows64_0 (1)"

result_files = sorted([
    f for f in os.listdir(foldx_path)
    if f.endswith("_results.txt") and f.count("_") == 2
])

def parse_result_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines()
        if len(lines) >= 5:
            return {
                "Protein": lines[0].split(":")[1].strip(),
                "Chain": lines[1].split(":")[1].strip(),
                "Mutation": lines[2].split(":")[1].strip(),
                "ΔΔG Value": float(lines[4].strip())
            }
        else:
            raise ValueError("Dosya biçimi beklenenden kısa.")

results = []
for filename in result_files:
    full_path = os.path.join(foldx_path, filename)
    try:
        results.append(parse_result_file(full_path))
    except Exception as e:
        print(f"{filename} dosyası atlandı. Hata: {e}")

if results:
    df = pd.DataFrame(results)
    excel_output_path = os.path.join(foldx_path, "results_total.xlsx")
    df.to_excel(excel_output_path, index=False)
    print(f"\n Tüm sonuçlar başarıyla kaydedildi: {excel_output_path}")
else:
    print("Uygun formatta hiç sonuç dosyası bulunamadı.")


1AYE_A31AG_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_A52AG_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_D36AA_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_E14AA_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_E20AG_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_E5AG_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_F37AL_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_F67AA_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_H53AA_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_I15AV_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_I23AV_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_I73AV_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
1AYE_I77AA_results.txt dosyası atlandı. Hata: Dosya biçimi beklenenden kısa.
